In [103]:
from rdflib import *
from rdflib.graph import Graph

In [104]:
modelet2 = Graph()
modelet2.parse("C:/Users/ghiot/Documents/TESI/ontologies/meaw.ttl", format="ttl")

<Graph identifier=N6cd2817186a3467fb3054bb1ad5c47a5 (<class 'rdflib.graph.Graph'>)>

In [105]:
meaw = Namespace("https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
modelet2.bind("meaw", meaw)
modelet2.bind("rdfs", rdfs)
modelet2.bind("owl", owl)
modelet2.bind("rdf", rdf)
modelet2.bind("xsd", xsd)

In [106]:
print(modelet2.serialize(format="ttl"))

@prefix : <http://www.semanticweb.org/ghiot/ontologies/2024/0/meaw_inf#> .
@prefix meaw: <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/ghiot/ontologies/2024/0/meaw_inf> a owl:Ontology ;
    rdfs:comment "First modelet of the Multimodal Emotio Annotation Ontology. Developed within my thesis research in collaboration with the netherlands Institute for Sound and Vision." .

meaw:50_anger a owl:NamedIndividual,
        meaw:EmotionRelation ;
    meaw:expressesEmotion meaw:frustration ;
    meaw:hasAuralTrigger meaw:stressed_syllables ;
    meaw:hasFrameContext meaw:disagreeing_with_opinion ;
    meaw:hasSpeaker meaw:adelheid_roosen ;
    meaw:hasStart "537" ;
    meaw:hasSubjectExpression meaw:experienced 

### Competency questions

#### 8. In what utterances emotions are appraised?

Expected result: 

The text of the utterances where emotions are appraised<br>
**"In de zaal zijn nogal wat mensen die zeggen tegen ons wij..."**<br>
**"Na jaren gekwetst door de laatste uitzending van adelheid roosen over de transseksuele ik...."**

In [107]:
cq8 = """
    PREFIX : <http://www.semanticweb.org/ghiot/ontologies/2024/0/meaw_inf#> 
    PREFIX meaw: <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
    SELECT ?utterance_text 
    WHERE {
        {
        ?emo_rel meaw:hasSubjectExpression meaw:appraised;
                :hasText ?utterance_text.
    }
    UNION
    {
    ?emo_rel meaw:hasSecondaryRelation ?sec_emo_rel;
            meaw:hasText ?utterance_text.
    ?sec_emo_rel meaw:hasSubjectExpression meaw:appraised.
    }
    }

"""

results = modelet2.query(cq8)

for row in results:
    print(row)
    print("\n")


(rdflib.term.Literal('In de zaal zijn nogal wat mensen die zeggen tegen ons wij zijn af en toe best gekwetst door wat er op de televisie getoond wordt u knikt ja u knikt.'),)


(rdflib.term.Literal('Na jaren gekwetst door de laatste uitzending van adelheid roosen over de transseksuele ik vond dat het in het belachelijke getrokken werd en ik denk dus dat er een heleboel mensen daardoor gekwetst zijn want werd in het seksuele vlak getrokken en dat vond ik dus niet goed.'),)




#### 9. In what utterances one can identify more than one basic emotion? What type of emotions are they?

Expected result: 

The text of the utterance, with the related emotion identified and the type of emotion<br>
- **"Nou ja ze begonnen dus mee met die bossen...", fear, present, experienced; annoyance, anger, past, experienced.**
- **"Na jaren gekwetst door de laatste uitzending van adelheid roosen...", sadness, present, experienced; sorrow, sadness, past, appraised**
- **"In de zaal zijn nogal wat mensen die zeggen tegen ons wij zijn af en...", neutrality; sorrow, sadness, past, appraised**

In [108]:
cq9 = """
    PREFIX : <http://www.semanticweb.org/ghiot/ontologies/2024/0/meaw_inf#> 
    PREFIX meaw: <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
    SELECT DISTINCT ?utterance_text ?emo1 ?b_emo1 ?exp_time1 ?exp_subj1 ?emo2 ?b_emo2 ?exp_time2 ?exp_subj2
    WHERE 
    {
        {
                ?emo_rel meaw:hasSecondaryRelation ?sec_emo_rel;
                        meaw:hasText ?utterance_text;
                        meaw:expressesEmotion ?emo1;
                        meaw:hasTimeExpression ?exp_time1;
                        meaw:hasSubjectExpression ?exp_subj1.

                ?sec_emo_rel meaw:expressesEmotion ?emo2;
                        meaw:hasTimeExpression ?exp_time2;
                        meaw:hasSubjectExpression ?exp_subj2.
                
                OPTIONAL {
                ?emo1 rdf:type ?emo_type1.
                FILTER NOT EXISTS { ?emo_type1 rdfs:subClassOf* meaw:BasicEmotion }
                ?b_emo1 meaw:includes ?emo1.
                }
                OPTIONAL {
                ?emo2 rdf:type ?emo_type2.
                FILTER NOT EXISTS { ?emo_type2 rdfs:subClassOf* meaw:BasicEmotion }
                ?b_emo2 meaw:includes ?emo2.
                }
        
        }
        UNION
        {
                ?emo_rel meaw:hasSecondaryRelation ?sec_emo_rel;
                        meaw:hasText ?utterance_text;
                        meaw:expressesEmotion ?emo1.
                
                ?sec_emo_rel meaw:expressesEmotion ?emo2;
                        meaw:hasTimeExpression ?exp_time2;
                        meaw:hasSubjectExpression ?exp_subj2.

                OPTIONAL {
                ?emo1 rdf:type ?emo_type1.
                FILTER NOT EXISTS { ?emo_type1 rdfs:subClassOf* meaw:BasicEmotion }
                ?b_emo1 meaw:includes ?emo1.
                }
                OPTIONAL {
                ?emo2 rdf:type ?emo_type2.
                FILTER NOT EXISTS { ?emo_type2 rdfs:subClassOf* meaw:BasicEmotion }
                ?b_emo2 meaw:includes ?emo2.
                }

                FILTER(?emo1 = meaw:neutrality)
        
        }

        }
"""

results = modelet2.query(cq9)

for row in results:
    print(row)
    print("\n")


(rdflib.term.Literal('Na jaren gekwetst door de laatste uitzending van adelheid roosen over de transseksuele ik vond dat het in het belachelijke getrokken werd en ik denk dus dat er een heleboel mensen daardoor gekwetst zijn want werd in het seksuele vlak getrokken en dat vond ik dus niet goed.'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sadness'), None, rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#present'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#experienced'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sorrow'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sadness'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#past'),

#### 10. What are the utterances where the triggers are either visual or aural? What are the triggers?

Expected result: 

The text of the utterances, the triggers<br>
- **"Vrouw en wat ik nu zeg maar zo snel...”, cover_mouth**<br>
- **"Tien minuten aan het woord moet...”, frowning**<br>
- **"Het moet kiezen voor de persoon in kwestie...”, frowning**<br>
- **"Wat bedoelt u met seksuele manier nou aan borst ...”, raise_eyebrow**<br>
- **"Ja ik woon vragen je hoorde zoveel kan je geen programma...”, headshake**<br>
- **"Tien minuten aan het woord moet...”, stressed_syllables**<br>
- **"Wat bedoelt u met seksuele manier nou aan...”, quick_bicker**<br>
- **"Dat zijn....”, tense_articulation**<br>

In [109]:
cq10 = """
    PREFIX : <http://www.semanticweb.org/ghiot/ontologies/2024/0/meaw_inf#> 
    PREFIX meaw: <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
    SELECT DISTINCT ?utt_text ?trigger

    WHERE {
                {
                ?emo_rel meaw:hasVisualTrigger ?trigger;
                        meaw:hasText ?utt_text.                        
                }
                UNION
                {
                ?emo_rel meaw:hasAuralTrigger ?trigger;
                        meaw:hasText ?utt_text.
                }

        }
"""

results = modelet2.query(cq10)

for row in results:
    print(row)
    print("\n")


(rdflib.term.Literal('Vrouw en wat ik nu zeg maar zo snel zou dus want dit nou dit voorbeeld aanhalen.'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#cover_mouth'))


(rdflib.term.Literal('Tien minuten aan het woord moet laten proberen een uitgangspunt te vinden waardoor je niet zoals in de meeste talkshows is dat je dus niet.'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#frowning'))


(rdflib.term.Literal('Het moet kiezen voor de persoon in kwestie of voor degenen die de talkshow leidt dat ik dus echt met alle respect wie acht minuten aan die persoon wilde geven als mensen dan kwetsend als kwetsend ervaren dan ben ik dus nou daar weer gekwetst en dan houdt het verhaal van mij op het uefa niets meer.'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#frowning'))


(rdflib.term.Literal('Wat bedoelt u met seksu

#### 11. What emotions are expressed in a dialogue?

Expected result: 

A list of emotions and their respective utterance.<br>
**"In de zaal zijn nogal wat mensen die zeggen tegen ons wij..."**<br>
**Anger, argumentativeness**

In [111]:
cq11 = """
    PREFIX : <http://www.semanticweb.org/ghiot/ontologies/2024/0/meaw_inf#> 
    PREFIX meaw: <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
SELECT ?text ?emo ?b_emo
WHERE {
  {
    SELECT ?emo_rel (COUNT(?speaker) as ?speakerCount) ?text ?emo ?b_emo
    WHERE {
      ?emo_rel meaw:hasSpeaker ?speaker ;
                meaw:hasText ?text;
                meaw:expressesEmotion ?emo.

          OPTIONAL {?b_emo meaw:includes ?emo} .
          
      
    }
    GROUP BY ?emo_rel
    HAVING (COUNT(?speaker) > 1)
  }
}
"""

results = modelet2.query(cq11)

for row in results:
    print(row)
    print("\n")


(rdflib.term.Literal('Wat bedoelt u met seksuele manier nou aan borst te raken en dergelijke ik denk dat een heleboel transseksuelen zich gekwetst zouden voelen ik weet het zeker van niet nou ik weet het dus niet maar wel voor mijn gevoel dus ja voor mijn gevoel dus helemaal niet ik mag dus mijn gevoel uit afstaan.'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#argumentativeness'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#anger'))


